# Electromagnetism — Maxwell's Equations & EM Waves

**SIIEA Quantum Engineering Curriculum**
- **Curriculum Days:** Days 197-224
- **License:** CC BY-NC-SA 4.0 | Siiea Innovations, LLC

---

In [ ]:
# Hardware detection — adapts simulations to your machine
import sys, os
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath("__file__")), ".."))
try:
    from hardware_config import HARDWARE, get_max_qubits
    print(f"Hardware: {HARDWARE['chip']} | {HARDWARE['memory_gb']} GB | Profile: {HARDWARE['profile']}")
    print(f"Max qubits: {get_max_qubits('safe')} (safe) / {get_max_qubits('max')} (max)")
except ImportError:
    print("hardware_config.py not found — using defaults")
    print("Run setup.sh from the repo root to generate it")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import sympy as sp
from scipy import integrate
from scipy.constants import epsilon_0, mu_0, c, e as e_charge, hbar

%matplotlib inline

plt.rcParams.update({
    "figure.figsize": (10, 7),
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "lines.linewidth": 2,
    "legend.fontsize": 11,
    "font.family": "serif",
    "figure.dpi": 120,
})
print("Imports ready — numpy, matplotlib, sympy, scipy loaded.")
print(f"Physical constants: ε₀ = {epsilon_0:.4e}, μ₀ = {mu_0:.4e}, c = {c:.4e} m/s")

\
## 1. Electric Fields: Coulomb's Law and Superposition

The electric field of a point charge $q$ at the origin is:

$$\vec{E}(\vec{r}) = \frac{1}{4\pi\epsilon_0}\frac{q}{r^2}\hat{r}$$

For multiple charges, the **superposition principle** gives:

$$\vec{E}(\vec{r}) = \frac{1}{4\pi\epsilon_0}\sum_i \frac{q_i}{|\vec{r} - \vec{r}_i|^2}\hat{r}_i$$

The **electric dipole** (charges $+q$ and $-q$ separated by distance $d$) produces
the characteristic field pattern that is fundamental to atomic physics and
molecular bonding.

In [ ]:
\
# Electric field visualization: point charge, dipole, quadrupole

def E_field(charges, positions, X, Y):
    '''Compute electric field from point charges at positions.'''
    Ex = np.zeros_like(X, dtype=float)
    Ey = np.zeros_like(Y, dtype=float)
    for q, (xq, yq) in zip(charges, positions):
        dx = X - xq
        dy = Y - yq
        r2 = dx**2 + dy**2
        r2 = np.maximum(r2, 1e-6)  # avoid singularity
        r3 = r2**1.5
        Ex += q * dx / r3
        Ey += q * dy / r3
    return Ex, Ey

x = np.linspace(-3, 3, 30)
y = np.linspace(-3, 3, 30)
X, Y = np.meshgrid(x, y)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

configs = [
    ("Point Charge (+)", [1], [(0, 0)]),
    ("Dipole (+/−)", [1, -1], [(-0.5, 0), (0.5, 0)]),
    ("Quadrupole", [1, -1, 1, -1], [(-1,-1), (-1,1), (1,-1), (1,1)]),
]

for ax, (title, charges, positions) in zip(axes, configs):
    Ex, Ey = E_field(charges, positions, X, Y)
    E_mag = np.sqrt(Ex**2 + Ey**2)
    E_mag = np.maximum(E_mag, 1e-10)
    # Normalize for uniform arrow length
    Ex_n = Ex / E_mag
    Ey_n = Ey / E_mag

    ax.quiver(X, Y, Ex_n, Ey_n, E_mag, cmap='inferno',
              norm=plt.Normalize(vmin=0, vmax=np.percentile(E_mag, 90)),
              alpha=0.8)
    for q, (xq, yq) in zip(charges, positions):
        color = 'red' if q > 0 else 'blue'
        ax.plot(xq, yq, 'o', color=color, markersize=12, markeredgecolor='black')
    ax.set_title(title, fontsize=13)
    ax.set_xlabel("x"); ax.set_ylabel("y")
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.2)

plt.tight_layout()
plt.show()
print("Quiver plots show field direction and magnitude (color).")
print("Dipole: field lines flow from + to −. Quadrupole: more complex topology.")

\
## 2. Gauss's Law

**Gauss's Law** (integral form):

$$\oint_S \vec{E} \cdot d\vec{A} = \frac{Q_{\text{enc}}}{\epsilon_0}$$

**Differential form:**

$$\nabla \cdot \vec{E} = \frac{\rho}{\epsilon_0}$$

The total electric flux through any closed surface equals the enclosed charge
divided by $\epsilon_0$. This is one of the four Maxwell equations and reflects
the fact that electric field lines **originate** on positive charges and
**terminate** on negative charges.

In [ ]:
\
# Numerical verification of Gauss's Law
# Compute the surface integral of E over a sphere surrounding a point charge

from scipy.integrate import dblquad

# Physical setup: charge Q at origin, integrate E over sphere of radius R
Q = 1.0  # Coulomb (we use 4πε₀ = 1 units for simplicity)
k = 1.0  # 1/(4πε₀) in natural units

# The flux should be Q/ε₀ = 4πkQ in these units

# Analytical: Φ = Q/ε₀ = 4πkQ
exact_flux = 4 * np.pi * k * Q

# Numerical integration using spherical coordinates
# E_r = kQ/R^2, dA = R^2 sin(θ) dθ dφ
# So E·dA = kQ sin(θ) dθ dφ (R cancels!)

radii = [0.5, 1.0, 2.0, 5.0, 10.0]

print("Gauss's Law Verification: ∮ E·dA = Q/ε₀")
print("=" * 60)
print(f"  Charge: Q = {Q}")
print(f"  Exact flux: 4πkQ = {exact_flux:.10f}")
print()

for R in radii:
    # Integrate kQ/R^2 * R^2 sin(θ) over θ∈[0,π], φ∈[0,2π]
    flux, error = dblquad(
        lambda theta, phi: k * Q * np.sin(theta),
        0, 2*np.pi,           # phi limits
        lambda phi: 0,         # theta lower
        lambda phi: np.pi,     # theta upper
    )
    rel_error = abs(flux - exact_flux) / exact_flux
    print(f"  R = {R:5.1f}:  Φ = {flux:.10f},  rel error = {rel_error:.2e}")

print(f"\nFlux is independent of radius — Gauss's law confirmed!")

\
## 3. Magnetic Fields: Biot-Savart Law

The magnetic field produced by a current element $I\,d\vec{l}$ is:

$$d\vec{B} = \frac{\mu_0}{4\pi}\frac{I\,d\vec{l} \times \hat{r}}{r^2}$$

For a **circular current loop** of radius $a$ carrying current $I$, the field
on the axis is:

$$B_z(z) = \frac{\mu_0 I a^2}{2(a^2 + z^2)^{3/2}}$$

At the center ($z=0$): $B = \mu_0 I / (2a)$.

This is the basis for understanding magnetic dipoles, MRI technology,
and the magnetic moments of atoms.

In [ ]:
\
# Magnetic field of a current loop via Biot-Savart numerical integration
# Compute B-field in the xz-plane for a loop of radius a in the xy-plane

def biot_savart_loop(a, I, points, N=1000):
    '''Compute B-field at given points from a circular loop
    of radius a, current I, centered at origin in xy-plane.
    Uses numerical integration of Biot-Savart law.'''
    mu0_over_4pi = 1e-7  # μ₀/(4π) in SI
    dphi = 2 * np.pi / N
    Bx = np.zeros(len(points))
    By = np.zeros(len(points))
    Bz = np.zeros(len(points))

    for i in range(N):
        phi = i * dphi
        # Current element position
        rl = np.array([a * np.cos(phi), a * np.sin(phi), 0.0])
        # Current element direction: dl = a*dphi * (-sin(phi), cos(phi), 0)
        dl = a * dphi * np.array([-np.sin(phi), np.cos(phi), 0.0])

        for j, r in enumerate(points):
            dr = r - rl
            dist = np.linalg.norm(dr)
            if dist < 1e-10:
                continue
            # dB = (μ₀/4π) I (dl × r̂) / r²
            dB = mu0_over_4pi * I * np.cross(dl, dr) / dist**3
            Bx[j] += dB[0]
            By[j] += dB[1]
            Bz[j] += dB[2]

    return Bx, By, Bz

# Compute on-axis field and compare with exact formula
a = 0.1   # loop radius (meters)
I = 1.0   # current (Amperes)
z_values = np.linspace(-0.3, 0.3, 50)
points_axis = [np.array([0, 0, z]) for z in z_values]

_, _, Bz_numerical = biot_savart_loop(a, I, points_axis, N=2000)

# Exact on-axis formula
mu0 = 4e-7 * np.pi
Bz_exact = mu0 * I * a**2 / (2 * (a**2 + z_values**2)**1.5)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: On-axis field comparison
ax = axes[0]
ax.plot(z_values * 100, Bz_numerical * 1e6, 'bo', markersize=4, label='Biot-Savart (numerical)')
ax.plot(z_values * 100, Bz_exact * 1e6, 'r-', label='Exact formula')
ax.set_xlabel("z (cm)", fontsize=12)
ax.set_ylabel("$B_z$ (μT)", fontsize=12)
ax.set_title(f"Current Loop: a={a*100} cm, I={I} A", fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Right: B-field in xz-plane (quiver plot)
ax2 = axes[1]
nx, nz = 15, 15
xg = np.linspace(-0.25, 0.25, nx)
zg = np.linspace(-0.25, 0.25, nz)
Xg, Zg = np.meshgrid(xg, zg)
points_grid = [np.array([x, 0, z]) for z in zg for x in xg]

Bx_grid, _, Bz_grid = biot_savart_loop(a, I, points_grid, N=500)
Bx_2d = Bx_grid.reshape(nz, nx)
Bz_2d = Bz_grid.reshape(nz, nx)
B_mag = np.sqrt(Bx_2d**2 + Bz_2d**2)
B_mag = np.maximum(B_mag, 1e-15)

ax2.quiver(Xg*100, Zg*100, Bx_2d/B_mag, Bz_2d/B_mag, np.log10(B_mag),
           cmap='viridis', alpha=0.8)
circle_x = a * np.cos(np.linspace(0, 2*np.pi, 100))
circle_z = a * np.sin(np.linspace(0, 2*np.pi, 100))
ax2.plot([-a*100, a*100], [0, 0], 'ro', markersize=10, label='Current loop (cross-section)')
ax2.set_xlabel("x (cm)", fontsize=12)
ax2.set_ylabel("z (cm)", fontsize=12)
ax2.set_title("B-field in xz-plane", fontsize=13)
ax2.set_aspect('equal')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.2)

plt.tight_layout()
plt.show()
print("Left: On-axis B-field matches exact formula to high precision.")
print("Right: Magnetic dipole field pattern visible in the xz-plane.")

\
## 4. Maxwell's Equations

The four Maxwell equations (in differential form) unify all of electromagnetism:

| Law | Equation | Physics |
|-----|----------|---------|
| Gauss (E) | $\nabla \cdot \vec{E} = \rho/\epsilon_0$ | Charges source E-field |
| Gauss (B) | $\nabla \cdot \vec{B} = 0$ | No magnetic monopoles |
| Faraday | $\nabla \times \vec{E} = -\partial \vec{B}/\partial t$ | Changing B creates E |
| Ampère-Maxwell | $\nabla \times \vec{B} = \mu_0\vec{J} + \mu_0\epsilon_0\partial\vec{E}/\partial t$ | Currents and changing E create B |

In vacuum ($\rho = 0$, $\vec{J} = 0$), combining Faraday and Ampère gives the **wave equation**:

$$\nabla^2 \vec{E} = \mu_0\epsilon_0\frac{\partial^2 \vec{E}}{\partial t^2}
= \frac{1}{c^2}\frac{\partial^2 \vec{E}}{\partial t^2}$$

where $c = 1/\sqrt{\mu_0\epsilon_0} \approx 3 \times 10^8$ m/s — **light is an electromagnetic wave**.

In [ ]:
\
# Maxwell's Equations: verify curl and divergence numerically
# using finite differences on a discrete grid

def divergence_2d(Fx, Fy, dx, dy):
    '''Compute divergence of 2D vector field using central differences.'''
    dFx_dx = np.gradient(Fx, dx, axis=1)
    dFy_dy = np.gradient(Fy, dy, axis=0)
    return dFx_dx + dFy_dy

def curl_z_2d(Fx, Fy, dx, dy):
    '''Compute z-component of curl for a 2D vector field.'''
    dFy_dx = np.gradient(Fy, dx, axis=1)
    dFx_dy = np.gradient(Fx, dy, axis=0)
    return dFy_dx - dFx_dy

# Create grid
N = 100
x = np.linspace(-3, 3, N)
y = np.linspace(-3, 3, N)
dx = x[1] - x[0]
dy = y[1] - y[0]
X, Y = np.meshgrid(x, y)

# Electric field of a point charge at origin: E = q*r_hat/r^2
q = 1.0
R2 = X**2 + Y**2
R2 = np.maximum(R2, 0.01)  # regularize
R = np.sqrt(R2)
Ex = q * X / R2**1.5
Ey = q * Y / R2**1.5

# Compute divergence (should be ~0 away from charge, peaked at origin)
div_E = divergence_2d(Ex, Ey, dx, dy)

# Magnetic field of straight wire (Biot-Savart): B = (μ₀I/2πr) φ_hat
Bx = -Y / R2
By = X / R2

# Curl of B (should be ~0 away from wire, peaked at origin)
curl_B = curl_z_2d(Bx, By, dx, dy)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Divergence of E
im1 = axes[0].imshow(div_E, extent=[-3,3,-3,3], origin='lower',
                      cmap='RdBu_r', vmin=-5, vmax=5)
axes[0].set_title(r"$\nabla \cdot \vec{E}$ (point charge)", fontsize=13)
axes[0].set_xlabel("x"); axes[0].set_ylabel("y")
plt.colorbar(im1, ax=axes[0], shrink=0.8)

# Curl of B
im2 = axes[1].imshow(curl_B, extent=[-3,3,-3,3], origin='lower',
                      cmap='RdBu_r', vmin=-5, vmax=5)
axes[1].set_title(r"$(\nabla \times \vec{B})_z$ (wire current)", fontsize=13)
axes[1].set_xlabel("x"); axes[1].set_ylabel("y")
plt.colorbar(im2, ax=axes[1], shrink=0.8)

plt.tight_layout()
plt.show()
print("Left: div(E) peaks at charge location — Gauss's law: ∇·E = ρ/ε₀")
print("Right: curl(B) peaks at wire location — Ampère's law: ∇×B = μ₀J")
print("Away from sources, both are approximately zero.")

\
## 5. Electromagnetic Wave Simulation — FDTD Method

The **Finite-Difference Time-Domain** (FDTD) method solves Maxwell's equations
directly on a grid. In 1D, the relevant equations are:

$$\frac{\partial E_y}{\partial t} = \frac{1}{\epsilon_0}\frac{\partial H_z}{\partial x}$$

$$\frac{\partial H_z}{\partial t} = \frac{1}{\mu_0}\frac{\partial E_y}{\partial x}$$

Using **Yee's staggered grid**, $E$ and $H$ are evaluated at alternating half-steps
in both space and time. This naturally enforces the correct phase relationship
between electric and magnetic fields in a propagating wave.

The FDTD method is widely used in photonics, antenna design, and
computational electrodynamics.

In [ ]:
\
# 1D FDTD Simulation of Electromagnetic Wave Propagation

# Grid parameters
Nx = 500         # number of spatial cells
Nt = 600         # number of time steps
dx = 1e-3        # spatial step (1 mm)
dt = dx / (2*c)  # time step (CFL condition: dt < dx/c)

# Fields (Yee staggering: Ey at integer points, Hz at half-integer)
Ey = np.zeros(Nx)
Hz = np.zeros(Nx)

# Courant number
S = c * dt / dx
print(f"FDTD Parameters: Nx={Nx}, Nt={Nt}, dx={dx*1e3} mm, dt={dt*1e12:.3f} ps")
print(f"Courant number S = c·dt/dx = {S:.4f} (must be < 1 for stability)")

# Source: Gaussian pulse
source_pos = 100
t0 = 40       # delay in time steps
spread = 12   # width in time steps

# Storage for animation frames
snapshots = []
snapshot_times = [0, 100, 200, 300, 400, 500]

# Run FDTD loop
for n in range(Nt):
    # Update Hz (magnetic field)
    Hz[:-1] += S * (Ey[1:] - Ey[:-1])

    # Gaussian source injection
    pulse = np.exp(-0.5 * ((n - t0) / spread)**2)
    Hz[source_pos] += pulse

    # Update Ey (electric field)
    Ey[1:] += S * (Hz[1:] - Hz[:-1])

    # Simple absorbing boundary conditions
    Ey[0] = 0
    Ey[-1] = 0

    if n in snapshot_times:
        snapshots.append((n, Ey.copy(), Hz.copy()))

# Plot snapshots
fig, axes = plt.subplots(3, 2, figsize=(14, 10))
for idx, (n, ey, hz) in enumerate(snapshots):
    ax = axes[idx // 2, idx % 2]
    x_mm = np.arange(Nx) * dx * 1e3
    ax.plot(x_mm, ey, 'b-', label='$E_y$', lw=1.5)
    ax.plot(x_mm, hz, 'r-', label='$H_z$', lw=1.5, alpha=0.7)
    ax.axvline(source_pos * dx * 1e3, color='green', ls='--', alpha=0.5, label='Source')
    ax.set_title(f"t = {n} steps ({n*dt*1e12:.1f} ps)", fontsize=11)
    ax.set_xlabel("x (mm)")
    ax.set_ylabel("Field amplitude")
    ax.legend(fontsize=9, loc='upper right')
    ax.set_ylim(-1.5, 1.5)
    ax.grid(True, alpha=0.3)

plt.suptitle("1D FDTD: EM Wave Propagation", fontsize=14, y=1.01)
plt.tight_layout()
plt.show()
print("FDTD simulation shows EM pulse propagating at speed c.")
print("E and H fields are in phase for a traveling wave, 90° out of phase in time.")

\
## 6. Special Relativity and the Lorentz Transformation

Maxwell's equations are **not** invariant under Galilean transformations — they
require the **Lorentz transformation**:

$$x' = \gamma(x - vt), \quad t' = \gamma\left(t - \frac{vx}{c^2}\right)$$

where $\gamma = 1/\sqrt{1 - v^2/c^2}$ is the Lorentz factor.

Key consequences:
- **Time dilation:** $\Delta t' = \gamma \Delta t$
- **Length contraction:** $\Delta x' = \Delta x / \gamma$
- **Mass-energy equivalence:** $E = mc^2$

The electromagnetic field tensor $F^{\mu\nu}$ unifies $\vec{E}$ and $\vec{B}$ into
a single object that transforms covariantly:

$$F^{\mu\nu} = \begin{pmatrix} 0 & -E_x/c & -E_y/c & -E_z/c \\
E_x/c & 0 & -B_z & B_y \\
E_y/c & B_z & 0 & -B_x \\
E_z/c & -B_y & B_x & 0 \end{pmatrix}$$

In [ ]:
\
# Lorentz Transformation Visualization

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# --- Left: Lorentz factor γ(v) ---
v_over_c = np.linspace(0, 0.999, 1000)
gamma = 1.0 / np.sqrt(1 - v_over_c**2)

ax = axes[0]
ax.plot(v_over_c, gamma, 'b-', lw=2)
ax.axhline(1, color='gray', ls='--', alpha=0.5)
for v_mark in [0.5, 0.9, 0.99]:
    g = 1/np.sqrt(1-v_mark**2)
    ax.plot(v_mark, g, 'ro', markersize=8)
    ax.annotate(f'v={v_mark}c\nγ={g:.2f}', (v_mark, g),
                textcoords="offset points", xytext=(-30, 10), fontsize=9)
ax.set_xlabel("v/c", fontsize=12)
ax.set_ylabel("γ", fontsize=12)
ax.set_title("Lorentz Factor", fontsize=13)
ax.set_ylim(0, 15)
ax.grid(True, alpha=0.3)

# --- Middle: Spacetime diagram with Lorentz-boosted axes ---
ax2 = axes[1]
beta = 0.5
gamma_b = 1/np.sqrt(1 - beta**2)

# Original axes
ax2.axhline(0, color='gray', lw=0.5)
ax2.axvline(0, color='gray', lw=0.5)

# Light cone
t_lc = np.linspace(-2.5, 2.5, 100)
ax2.plot(t_lc, t_lc, 'y-', lw=2, label='Light cone (x=ct)')
ax2.plot(t_lc, -t_lc, 'y-', lw=2)

# Boosted t' axis: x = βct (i.e., x = β·t in ct units)
ax2.plot(beta * t_lc, t_lc, 'r--', lw=2, label=f"t' axis (β={beta})")
# Boosted x' axis: ct = βx
ax2.plot(t_lc, beta * t_lc, 'b--', lw=2, label=f"x' axis (β={beta})")

# World line of stationary object
ax2.plot([0, 0], [-2.5, 2.5], 'k-', lw=2, label='Stationary observer')

ax2.set_xlabel("x (units of ct)", fontsize=12)
ax2.set_ylabel("ct", fontsize=12)
ax2.set_title("Minkowski Diagram", fontsize=13)
ax2.set_xlim(-2.5, 2.5)
ax2.set_ylim(-2.5, 2.5)
ax2.set_aspect('equal')
ax2.legend(fontsize=9, loc='upper left')
ax2.grid(True, alpha=0.3)

# --- Right: E and B field transformation ---
ax3 = axes[2]
# Pure E-field in rest frame → mixed E,B in moving frame
# E'_y = γ(E_y - vB_z), B'_z = γ(B_z - vE_y/c^2)
E0 = 1.0  # E_y in rest frame
B0 = 0.0  # B_z in rest frame

betas = np.linspace(0, 0.99, 100)
gammas = 1/np.sqrt(1 - betas**2)
Ey_prime = gammas * (E0 - betas * c * B0)
Bz_prime = gammas * (B0 - betas * E0 / c)

ax3.plot(betas, Ey_prime / E0, 'b-', lw=2, label="$E'_y / E_0$")
ax3.plot(betas, Bz_prime * c / E0, 'r-', lw=2, label="$cB'_z / E_0$")
ax3.set_xlabel("v/c", fontsize=12)
ax3.set_ylabel("Transformed field / $E_0$", fontsize=12)
ax3.set_title("Field Transformation (pure E → E+B)", fontsize=13)
ax3.legend(fontsize=11)
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
print("Left: Lorentz factor diverges as v → c.")
print("Middle: Boosted axes tilt toward the light cone.")
print("Right: A pure E-field gains a B-component in a moving frame — E and B are unified!")

\
## 7. Quantum Mechanics Connection: Quantized EM Field

### Photons as Quantized EM Waves

The electromagnetic field is quantized by promoting the field amplitudes
to operators. Each mode of frequency $\omega$ becomes a quantum harmonic oscillator:

$$\hat{H} = \hbar\omega\left(\hat{a}^\dagger\hat{a} + \frac{1}{2}\right)$$

where $\hat{a}^\dagger$ creates a photon and $\hat{a}$ destroys one. Energy levels:

$$E_n = \hbar\omega\left(n + \frac{1}{2}\right)$$

### Minimal Coupling

The interaction of a charged particle with the EM field introduces
the **minimal coupling** replacement:

$$\hat{\vec{p}} \to \hat{\vec{p}} - q\vec{A}$$

leading to the Hamiltonian for the hydrogen atom:

$$\hat{H} = \frac{1}{2m}\left(\hat{\vec{p}} - \frac{e}{c}\vec{A}\right)^2 - \frac{e^2}{4\pi\epsilon_0 r}$$

### Vacuum Fluctuations

Even in the ground state ($n=0$), the zero-point energy
$E_0 = \hbar\omega/2$ gives rise to **vacuum fluctuations** — measurable
via the Casimir effect and the Lamb shift.

In [ ]:
\
# Quantized EM field: Fock states and field expectation values

from scipy.special import hermite, factorial

def coherent_state_prob(n_max, alpha):
    '''Photon number distribution for coherent state |alpha>.'''
    ns = np.arange(n_max)
    mean_n = abs(alpha)**2
    # P(n) = e^{-|α|²} |α|^{2n} / n!
    log_P = -mean_n + 2*ns*np.log(abs(alpha)) - np.array([np.sum(np.log(np.arange(1, k+1))) if k > 0 else 0 for k in ns])
    return ns, np.exp(log_P)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# --- Left: Energy levels of quantized EM mode ---
ax = axes[0]
omega = 1.0  # normalized
n_levels = 8
for n in range(n_levels):
    E = omega * (n + 0.5)
    ax.hlines(E, 0.2, 0.8, colors='blue', linewidth=2)
    ax.text(0.85, E, f'$|{n}\\rangle$, $E_{n}={E:.1f}\\hbar\\omega$',
            fontsize=10, va='center')
ax.set_xlim(0, 2.0)
ax.set_ylim(0, n_levels + 0.5)
ax.set_ylabel("$E / \\hbar\\omega$", fontsize=12)
ax.set_title("Quantized EM Mode Energy Levels", fontsize=13)
ax.set_xticks([])
ax.axhline(0.5, color='red', ls='--', alpha=0.5, label='Zero-point energy')
ax.legend(fontsize=10)

# --- Middle: Photon number distribution for coherent states ---
ax2 = axes[1]
for alpha in [1.0, 2.0, 3.0]:
    ns, probs = coherent_state_prob(20, alpha)
    ax2.bar(ns + 0.15*(alpha-2), probs, width=0.15, alpha=0.7,
            label=f'$|\\alpha={alpha}\\rangle$, $\\langle n \\rangle={alpha**2:.0f}$')
ax2.set_xlabel("Photon number $n$", fontsize=12)
ax2.set_ylabel("$P(n)$", fontsize=12)
ax2.set_title("Coherent State Photon Distributions", fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

# --- Right: E-field quadrature for vacuum, coherent, and squeezed ---
ax3 = axes[2]
x = np.linspace(-5, 5, 500)

# Vacuum state: Gaussian centered at 0
psi_vac = np.exp(-x**2 / 2) / np.pi**0.25
ax3.fill_between(x, psi_vac**2, alpha=0.3, color='blue', label='Vacuum $|0\\rangle$')

# Coherent state |α=2⟩: displaced Gaussian
alpha = 2.0
psi_coh = np.exp(-(x - np.sqrt(2)*alpha)**2 / 2) / np.pi**0.25
ax3.fill_between(x, psi_coh**2, alpha=0.3, color='red', label=f'Coherent $|\\alpha={alpha}\\rangle$')

# Squeezed state: narrower Gaussian
r = 0.8  # squeezing parameter
sigma_sq = np.exp(-2*r)
psi_sq = (1/(np.pi * sigma_sq))**0.25 * np.exp(-x**2 / (2*sigma_sq))
ax3.fill_between(x, psi_sq**2, alpha=0.3, color='green', label=f'Squeezed ($r={r}$)')

ax3.set_xlabel("Field quadrature $X$", fontsize=12)
ax3.set_ylabel("$|\\psi(X)|^2$", fontsize=12)
ax3.set_title("Quantum Field States", fontsize=13)
ax3.legend(fontsize=10)
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
print("Left: Equally spaced energy levels — each quantum is one photon ℏω.")
print("Middle: Coherent states have Poissonian photon statistics — closest to classical.")
print("Right: Squeezed states have reduced uncertainty in one quadrature — used in LIGO!")

In [ ]:
\
# Hydrogen atom: from classical EM to quantum energy levels
# The hydrogen atom is where EM and QM meet

from scipy.constants import m_e, e as e_charge, hbar, epsilon_0, alpha

# Bohr model energy levels: E_n = -13.6 eV / n²
E_1 = -m_e * e_charge**4 / (2 * (4*np.pi*epsilon_0)**2 * hbar**2)  # in Joules
E_1_eV = E_1 / e_charge  # convert to eV

print("Hydrogen Atom Energy Levels (Bohr Model)")
print("=" * 55)
print(f"  Ground state energy: E₁ = {E_1_eV:.4f} eV")
print(f"  Fine structure constant: α = {alpha:.6f}")
print(f"  Bohr radius: a₀ = {hbar/(m_e * e_charge**2 / (4*np.pi*epsilon_0)) * 1e10:.4f} Å")
print()

fig, ax = plt.subplots(figsize=(10, 7))

n_max = 7
for n in range(1, n_max+1):
    E_n = E_1_eV / n**2
    ax.hlines(E_n, 0.5, 4.5, colors='blue', linewidth=2)
    ax.text(4.6, E_n, f'$n={n}$: {E_n:.3f} eV', fontsize=10, va='center')

# Draw some transitions (Lyman, Balmer, Paschen series)
transitions = {
    'Lyman': [(2,1,'violet'), (3,1,'indigo'), (4,1,'blue')],
    'Balmer': [(3,2,'red'), (4,2,'cyan'), (5,2,'green')],
    'Paschen': [(4,3,'orange'), (5,3,'brown')],
}

x_pos = {('Lyman', 0): 1.0, ('Lyman', 1): 1.3, ('Lyman', 2): 1.6,
         ('Balmer', 0): 2.2, ('Balmer', 1): 2.5, ('Balmer', 2): 2.8,
         ('Paschen', 0): 3.4, ('Paschen', 1): 3.7}

for series, trans in transitions.items():
    for i, (n_upper, n_lower, color) in enumerate(trans):
        E_upper = E_1_eV / n_upper**2
        E_lower = E_1_eV / n_lower**2
        xp = x_pos.get((series, i), 2.0)
        ax.annotate('', xy=(xp, E_lower), xytext=(xp, E_upper),
                    arrowprops=dict(arrowstyle='->', color=color, lw=1.5))
        # Photon energy
        dE = E_upper - E_lower  # negative because both are negative
        wavelength = 1240 / abs(dE)  # nm (from E = hc/λ with E in eV)

ax.set_xlim(0, 6)
ax.set_ylabel("Energy (eV)", fontsize=12)
ax.set_title("Hydrogen Energy Levels and Spectral Series", fontsize=14)
ax.set_xticks([1.3, 2.5, 3.55])
ax.set_xticklabels(['Lyman\n(UV)', 'Balmer\n(visible)', 'Paschen\n(IR)'], fontsize=10)
ax.grid(True, alpha=0.2, axis='y')

plt.tight_layout()
plt.show()

# Print transition wavelengths
print("\nSpectral Lines:")
for series, trans in transitions.items():
    print(f"  {series} series:")
    for n_upper, n_lower, _ in trans:
        dE = abs(E_1_eV / n_lower**2 - E_1_eV / n_upper**2)
        lam = 1240 / dE
        print(f"    {n_upper} → {n_lower}: ΔE = {dE:.3f} eV, λ = {lam:.1f} nm")

\
## Summary

| Topic | Key Result |
|-------|-----------|
| Coulomb's law | $\vec{E} = \frac{1}{4\pi\epsilon_0}\frac{q}{r^2}\hat{r}$ |
| Gauss's law | $\oint \vec{E}\cdot d\vec{A} = Q_{\rm enc}/\epsilon_0$ |
| Biot-Savart | $d\vec{B} = \frac{\mu_0}{4\pi}\frac{I\,d\vec{l}\times\hat{r}}{r^2}$ |
| Maxwell's eqs | 4 equations unify E, B, charges, currents |
| EM waves | $c = 1/\sqrt{\mu_0\epsilon_0}$ — light is EM radiation |
| Lorentz transform | $x'=\gamma(x-vt)$, $t'=\gamma(t-vx/c^2)$ |
| Quantized field | $\hat{H} = \hbar\omega(\hat{a}^\dagger\hat{a}+\frac{1}{2})$ |
| Hydrogen | $E_n = -13.6\,\text{eV}/n^2$ from EM + QM |

**Next:** Month 09 — Functional Analysis and Hilbert Spaces

---
*SIIEA Quantum Engineering Curriculum — CC BY-NC-SA 4.0*